In [2]:
import pandas as pd

In [3]:
ratings = pd.read_csv('data/movielens/ratings.csv')
movies = pd.read_csv('data/movielens/movies.csv')
movies.set_index('movieId',inplace=True)
movies.head(5)

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [21]:
print ('ratings')
ratings.head()

ratings


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


# Independent Recommendation
In this case we are only interested in top 10 movies 

In [4]:
top5movies = ratings[['movieId','rating']].groupby('movieId').mean().sort_values('rating').tail(5)
top5movies.head()
top5movies['moviename'] = top5movies.index.map(lambda mid : movies.loc[mid,'title'])



In [47]:
top5movies

,rating,moviename
movieId,,
3165,5.0,Boiling Point (1993)
27792,5.0,"Saddest Music in the World, The (2003)"
3202,5.0,Even Dwarfs Started Small (Auch Zwerge haben k...
3216,5.0,"Vampyros Lesbos (Vampiras, Las) (1971)"
163949,5.0,The Beatles: Eight Days a Week - The Touring Y...


## Collaborative Filtering - Item Item Similarity

In [5]:
movie_users = ratings[['movieId','rating','userId']].groupby('movieId')['userId'].apply(list)
movie_users


movieId
1         [7, 9, 13, 15, 19, 20, 23, 26, 30, 37, 43, 44,...
2         [15, 19, 30, 32, 41, 47, 48, 49, 61, 68, 69, 7...
3         [5, 19, 25, 44, 70, 75, 85, 100, 110, 112, 115...
4         [19, 113, 128, 168, 182, 239, 391, 460, 461, 5...
5         [15, 18, 44, 55, 69, 70, 85, 98, 110, 111, 113...
6         [15, 17, 18, 19, 23, 24, 30, 34, 36, 39, 44, 5...
7         [18, 19, 55, 70, 79, 80, 87, 88, 100, 110, 111...
8                                  [30, 113, 128, 182, 614]
9         [18, 19, 55, 70, 95, 110, 113, 182, 247, 254, ...
10        [2, 4, 7, 15, 19, 21, 39, 47, 50, 56, 73, 74, ...
11        [15, 19, 23, 30, 57, 58, 59, 63, 68, 86, 88, 9...
12        [67, 70, 91, 157, 199, 247, 252, 284, 294, 358...
13                 [182, 213, 287, 402, 516, 518, 564, 595]
14        [15, 18, 19, 30, 70, 86, 102, 233, 256, 266, 2...
15        [73, 213, 311, 390, 396, 409, 496, 564, 598, 6...
16        [15, 19, 23, 30, 36, 38, 39, 56, 60, 63, 67, 7...
17        [2, 9, 15, 18, 44, 57,

In [80]:
def find_similar(movieId):
    movie_users_cp = movie_users
    users = movie_users_cp[movieId]
    commons = movie_users_cp.map(lambda x : len(set(x).intersection(set(users))))
    #print (commons.sort_values()[-6:-1])
    top_results=commons.sort_values()[-2:-7:-1]
    print ('You searched : ',movies.loc[movieId,'title'])
    print ('\n Top Results : ')
    for topId,count in top_results.iteritems():
        print (count,movies.loc[topId,'title'])
    
find_similar(1200)

You searched :  Aliens (1986)

 Top Results : 
101 Star Wars: Episode V - The Empire Strikes Back (1980)
100 Star Wars: Episode IV - A New Hope (1977)
94 Back to the Future (1985)
93 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
91 Matrix, The (1999)


In [101]:
actual = ratings.groupby('movieId').agg({'userId': lambda x : list(x),
                                        'rating': lambda x : list(x)})
actual_group = ratings.groupby('movieId')

In [100]:
actual.head(5)

,rating,userId
movieId,,
1,"[3.0, 4.0, 5.0, 2.0, 3.0, 3.5, 3.0, 5.0, 4.0, ...","[7, 9, 13, 15, 19, 20, 23, 26, 30, 37, 43, 44,..."
2,"[2.0, 3.0, 2.0, 4.0, 3.5, 5.0, 3.5, 5.0, 3.5, ...","[15, 19, 30, 32, 41, 47, 48, 49, 61, 68, 69, 7..."
3,"[4.0, 3.0, 3.0, 5.0, 5.0, 3.0, 2.0, 4.0, 4.0, ...","[5, 19, 25, 44, 70, 75, 85, 100, 110, 112, 115..."
4,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.5, 1.5, ...","[19, 113, 128, 168, 182, 239, 391, 460, 461, 5..."
5,"[4.5, 3.0, 3.0, 3.0, 5.0, 5.0, 3.0, 3.0, 4.0, ...","[15, 18, 44, 55, 69, 70, 85, 98, 110, 111, 113..."


In [115]:
user_df1 = actual_group.get_group(1)[['userId','rating']].set_index('userId')
user_df1['rating'][:5]

userId
7     3.0
9     4.0
13    5.0
15    2.0
19    3.0
Name: rating, dtype: float64

In [139]:
import numpy as np
def get_similarity(movieId1,movieId2):
    user_df1 = actual_group.get_group(movieId1)[['userId','rating']].set_index('userId')
    user_series1=user_df1['rating']
    user_df2 = actual_group.get_group(movieId2)[['userId','rating']].set_index('userId')
    user_series2=user_df2['rating']
    combined_df = pd.DataFrame({'movie1':user_series1,'movie2':user_series2})
    combined_df.dropna(inplace=True)
    return np.mean(np.array(combined_df['movie1']-2.5)*np.array(combined_df['movie2']-2.5))
 
sim_dict={}
def get_similar(movieId):
    sim_dict={key:get_similarity(movieId,key) for key in actual_group.groups.keys()}
    print(pd.Series(sim_dict).dropna().sort_values()[-5:])

In [140]:
get_similar(1)

C:\Users\jaley\AppData\Local\conda\conda\envs\condaenv\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\jaley\AppData\Local\conda\conda\envs\condaenv\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


95313    6.25
95377    6.25
264      6.25
36931    6.25
3718     6.25
dtype: float64
